<a href="https://colab.research.google.com/github/IsfaquethedataAnalyst/Data_Analyst/blob/main/Supply_Chain_Network_Optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Add Libraries
check = c("lpSolveAPI","data.table") %in% installed.packages()
if(!check[1]) install.packages("lpSolveAPI")
if(!check[2]) install.packages("data.table")
library(lpSolveAPI); library(data.table)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
data <- read.csv("/content/Distance Matrix.csv")

In [3]:
# Distance Matrix
Dist = fread("Distance Matrix.csv")
Dist$Index = 1:nrow(Dist)

In [5]:
# Factories & Capacities
FT = data.table(Factory = unique(Dist$Source)[1:2],
                Capacity = c(150000, 200000))


In [6]:
# Distribution Centres & Throughputs
DC = data.table(
  DistCentre = unique(Dist$Source)[3:6],
  Throughput = c(70000, 50000,100000,40000))

In [7]:
# Customers & Demands
CT = data.table(Customer = unique(Dist$Destination)[5:10],
                Demand = c(50000,10000,40000,35000,60000,20000))


In [8]:
print(Dist)
print(FT)
print(DC)
print(CT)

        Source Destination Distance Index
        <char>      <char>    <int> <int>
 1:  Liverpool   Newcastle      178     1
 2:  Liverpool  Birmingham       99     2
 3:  Liverpool      London      221     3
 4:  Liverpool      Exeter      251     4
 5:   Brighton   Newcastle      345     5
 6:   Brighton  Birmingham      176     6
 7:   Brighton      London       65     7
 8:   Brighton      Exeter      206     8
 9:  Newcastle    Carlisle       60     9
10:  Newcastle  Darlington       37    10
11:  Newcastle   Sheffield      130    11
12:  Newcastle   Cambridge      236    12
13:  Newcastle      Oxford      258    13
14:  Newcastle       Truro      458    14
15: Birmingham    Carlisle      193    15
16: Birmingham  Darlington      172    16
17: Birmingham   Sheffield       91    17
18: Birmingham   Cambridge       99    18
19: Birmingham      Oxford       79    19
20: Birmingham       Truro      249    20
21:     London    Carlisle      307    21
22:     London  Darlington      24

# Task 2 - Problem Formulation & Objective Function

In [9]:
# lpSolve Object (0 constraints, nrow(Dist) variables)
m <- make.lp(0,nrow(Dist))

In [10]:
# Cost Function - Minimize Cost = Qty Shipped x Distance
# set.objfn(model, vector of coefficients of all Xs)
set.objfn(m,Dist$Distance)

In [11]:
# Exercise 2:
# Display the Customer Demand (figures only)
# Use the $ to get the column from dataframe

In [12]:
CT$Demand

[1] 50000 10000 40000 35000 60000 20000

In [13]:
# add.constraint(model, coeff, sign, rhs, indexes)

xcoef = rep(1, nrow(Dist[Source == "Liverpool"]))
xpos   = Dist[Source == "Liverpool"]$Index
xrhs  = FT[Factory == "Liverpool"]$Capacity
add.constraint(m,xcoef,"<=", xrhs, xpos)

xcoef = rep(1, nrow(Dist[Source == "Brighton"]))
xpos   = Dist[Source == "Brighton"]$Index
xrhs  = FT[Factory == "Brighton"]$Capacity
add.constraint(m,xcoef,"<=", xrhs, xpos)

In [14]:
# Extract index values for the above

Dist[Source == "Newcastle"]
Dist[Source == "Newcastle"]$Index

Source,Destination,Distance,Index
<chr>,<chr>,<int>,<int>
Newcastle,Carlisle,60,9
Newcastle,Darlington,37,10
Newcastle,Sheffield,130,11
Newcastle,Cambridge,236,12
Newcastle,Oxford,258,13
Newcastle,Truro,458,14


[1]  9 10 11 12 13 14

In [15]:
# Task 4 - Distribution Centre Constraints
#          Qty out of DC - Qty into DC = 0
#          Qty out of DC <= DC Throughput Limit

In [16]:
# Qty out of DC - Qty into DC = 0
for(i in DC$DistCentre) {
  xcoef = c(rep(1, nrow(Dist[Source == i])),
            rep(-1, nrow(Dist[Destination == i])))
  xpos  = c(Dist[Source == i]$Index,
            Dist[Destination == i]$Index)
  add.constraint(m,xcoef,"=", 0, xpos)
}

In [19]:
set.type(m, Dist$Index,"integer")

In [20]:
# Solve & Get Solution
x = proc.time()
lp.control(m, timeout = 10)

solve(m)
proc.time() - x

get.objective(m)

out = get.variables(m)
Dist$Qty = out

Dist[Qty != 0]

$anti.degen
[1] "fixedvars" "stalling" 

$basis.crash
[1] "none"

$bb.depthlimit
[1] -50

$bb.floorfirst
[1] "automatic"

$bb.rule
[1] "pseudononint" "greedy"       "dynamic"      "rcostfixing" 

$break.at.first
[1] FALSE

$break.at.value
[1] -1e+30

$epsilon
      epsb       epsd      epsel     epsint epsperturb   epspivot 
     1e-10      1e-09      1e-12      1e-07      1e-05      2e-07 

$improve
[1] "dualfeas" "thetagap"

$infinite
[1] 1e+30

$maxpivot
[1] 250

$mip.gap
absolute relative 
   1e-11    1e-11 

$negrange
[1] -1e+06

$obj.in.basis
[1] TRUE

$pivoting
[1] "devex"    "adaptive"

$presolve
[1] "none"

$scalelimit
[1] 5

$scaling
[1] "geometric"   "equilibrate" "integers"   

$sense
[1] "minimize"

$simplextype
[1] "dual"   "primal"

$timeout
[1] 10

$verbose
[1] "neutral"

[1] 0

   user  system elapsed 
  0.024   0.001   0.026 

[1] 39135000

Source,Destination,Distance,Index,Qty
<chr>,<chr>,<int>,<int>,<dbl>
Liverpool,Newcastle,178,1,60000
Liverpool,Birmingham,99,2,40000
Brighton,London,65,7,95000
Brighton,Exeter,206,8,20000
Newcastle,Carlisle,60,9,50000
Newcastle,Darlington,37,10,10000
Birmingham,Sheffield,91,17,40000
London,Cambridge,58,24,35000
London,Oxford,56,25,60000
